# Analysis of Belle2 Data

In [23]:
import os, sys, pprint
from root_pandas import read_root
import pandas as pd
import numpy as np
import seaborn as sns
import tensorflow as tf
from tensorflow import keras
from sklearn import preprocessing

In [2]:
!ls -l DATA

total 56200
-rwxr--r-- 1 philipp philipp   584128 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_ccbar_0.root
-rwxr--r-- 1 philipp philipp   117621 May 14 14:58 B0_etapr-eta-gg2pi_KS-pi+pi-_output_charged_17.root
-rwxr--r-- 1 philipp philipp   155084 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_0.root
-rwxr--r-- 1 philipp philipp   177149 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_1.root
-rwxr--r-- 1 philipp philipp   183977 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_2.root
-rwxr--r-- 1 philipp philipp   176779 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_3.root
-rwxr--r-- 1 philipp philipp   185028 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_4.root
-rwxr--r-- 1 philipp philipp   141664 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_5.root
-rwxr--r-- 1 philipp philipp   192110 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_6.root
-rwxr--r-- 1 philipp philipp   179731 May 14 14:52 B0_etapr-eta-gg2pi_KS-pi+pi-_output_DC_7.root
-rwxr--r-

## Reading and Prepairing Data

In [141]:
Signal = read_root('../DATA/B0_etapr-eta-gg2pi_KS-pi+pi-_output_sgnMC9_BGx1_0.root', 'B0')
DD = read_root('../DATA/B0_etapr-eta-gg2pi_KS-pi+pi-_output_ddbar_0.root', 'B0')

__Branches, organized by Categories__

In [34]:
General = ['exp_no', 'run_no', 'evt_no', 'nCands', 'iCand']
Masses = ['B0_M', 'B0_ErrM', 'B0_SigM', 'B0_K_S0_M', 'B0_K_S0_ErrM', 'B0_K_S0_SigM', 'B0_etap_M', 'B0_etap_ErrM', 
          'B0_etap_SigM', 'B0_etap_eta_M', 'B0_etap_eta_ErrM', 'B0_etap_eta_SigM']
Kinetics = ['B0_P', 'B0_P4', 'B0_deltae', 'B0_mbc', 'B0_etap_P', 'B0_etap_P4', 
            'B0_etap_eta_P', 'B0_etap_eta_P4', 'B0_etap_eta_gamma0_P', 'B0_etap_eta_gamma0_P4',
            'B0_etap_eta_gamma1_P', 'B0_etap_eta_gamma1_P4', 'B0_etap_pi0_P', 'B0_etap_pi0_P4',
            'B0_etap_pi1_P', 'B0_etap_pi1_P4', 'B0_K_S0_P', 'B0_K_S0_P4']
DecayAngles = ['B0_decayAngle__bo0__bc', 'B0_decayAngle__bo1__bc', 'B0_etap_decayAngle__bo0__bc',
               'B0_etap_decayAngle__bo1__bc', 'B0_etap_decayAngle__bo2__bc']
Positions = ['B0_X', 'B0_ErrX', 'B0_Y', 'B0_ErrY', 'B0_Z', 'B0_ErrZ', 'B0_Rho',  
             'B0_etap_X', 'B0_etap_ErrX', 'B0_etap_Y', 'B0_etap_ErrY', 
             'B0_etap_Z', 'B0_etap_ErrZ', 'B0_etap_Rho',
             'B0_etap_eta_X', 'B0_etap_eta_ErrX', 'B0_etap_eta_Y',
             'B0_etap_eta_ErrY', 'B0_etap_e ta_Z', 'B0_etap_eta_ErrZ', 'B0_etap_eta_Rho',
             'B0_etap_pi0_X', 'B0_etap_pi0_ErrX', 'B0_etap_pi0_Y', 'B0_etap_pi0_ErrY', 
             'B0_etap_pi0_Z', 'B0_etap_pi0_ErrZ', 'B0_etap_pi0_Rho', 
             'B0_etap_pi1_X', 'B0_etap_pi1_ErrX', 'B0_etap_pi1_Y', 'B0_etap_pi1_ErrY', 
             'B0_etap_pi1_Z', 'B0_etap_pi1_ErrZ', 'B0_etap_pi1_Rho', 
             'B0_K_S0_X', 'B0_K_S0_ErrX', 'B0_K_S0_Y', 'B0_K_S0_ErrY', 'B0_K_S0_Z',
             'B0_K_S0_ErrZ', 'B0_K_S0_Rho', 
             'B0_cosAngleBetweenMomentumAndVertexVector', 'B0_distance', 'B0_significanceOfDistance',
             'B0_dr', 'B0_etap_pi0_dr', 'B0_etap_pi1_dr', 'B0_K_S0_dr']
Vertex = ['B0_VtxPvalue', 'B0_VtxProd', 'B0_VtxProdCov', 'B0_etap_VtxPvalue', 'B0_etap_VtxProd', 'B0_etap_VtxProdCov',
          'B0_etap_eta_VtxPvalue', 'B0_etap_eta_VtxProd', 'B0_etap_VtxProdCov', 
          'B0_etap_pi0_VtxPvalue', 'B0_etap_pi0_VtxProd', 'B0_etap_pi0_VtxProdCov', 
          'B0_etap_pi1_VtxPvalue', 'B0_etap_pi1_VtxProd', 'B0_etap_pi1_VtxProdCov', 
          'B0_K_S0_VtxPvalue', 'B0_K_S0_VtxProd', 'B0_K_S0_VtxProdCov']
Continuum_Supression = ['B0_CSMVA', 'B0_TrCSMVA']

In [97]:
Signal[Masses]

,B0_M,B0_ErrM,B0_SigM,B0_K_S0_M,B0_K_S0_ErrM,B0_K_S0_SigM,B0_etap_M,B0_etap_ErrM,B0_etap_SigM,B0_etap_eta_M,B0_etap_eta_ErrM,B0_etap_eta_SigM
0,5.299881,0.010961,1.856670,0.500407,0.004492,0.621686,0.956255,0.002487,-0.565081,0.541614,0.012183,-0.512100
1,5.288764,0.034654,0.266445,0.496696,0.004302,-0.213392,0.959869,0.002066,1.069145,0.576153,0.025517,1.109071
2,5.303726,0.024997,0.967938,0.501817,0.004086,1.028760,0.962386,0.005514,0.857073,0.556313,0.010071,0.840091
3,5.300632,0.029411,0.717458,0.513165,0.011597,1.340991,0.956870,0.004230,-0.186780,0.568429,0.017586,1.170032
4,5.290727,0.014904,0.751298,0.498159,0.002639,0.206475,0.957299,0.001756,-0.205384,0.558190,0.012196,0.847595
5,5.249694,0.030067,-0.992315,0.497600,0.004264,-0.003369,0.958983,0.004562,0.290003,0.533261,0.012052,-1.210761
6,5.279331,0.019945,-0.009969,0.497564,0.003823,-0.013096,0.961598,0.005012,0.785671,0.533312,0.009823,-1.480309
7,5.255838,0.022732,-1.042220,0.492344,0.003371,-1.563578,0.953827,0.003155,-1.215142,0.545160,0.013446,-0.200311
8,5.281927,0.019656,0.121929,0.500738,0.005999,0.520837,0.961539,0.003692,1.050805,0.573878,0.013334,1.951728
9,5.276699,0.019774,-0.143170,0.494526,0.002019,-1.530039,0.950948,0.003950,-1.699346,0.514371,0.010224,-3.274710


#### Choosing the best candidates in each event

Combinatorics in the reconstruction process often leads to events with multiple candidates, where we carefully have to choose the best without introducing significant bias. To do this we have two possible methods: 

- Choosing the candidate with the reconstructed mass closest to the nominal value
- Choosing the candidate with the best vertex fit

If we were only using the mass we would bias our data towards a very narrow mass peak in cases where two candidates are close to the nominal value. Therefore I suggest using the masses for a generous cut (e.g. $5\sigma$) and then using vertex information to pick the best candidate. 

##### Masses
Nominal masses with errors in GeV $ [M_{nominal}, \sigma_{M_{nominal}}] $ according to the [Particle Data Group](http://pdg.lbl.gov/2019/listings/contents_listings.html)

In events with more than one candidate we can use this to perform a rough cut on candidates requirering, that 

$$ 
M_{reconstructed} \in 
    \left[ M_{nominal} - 5 \times \left( \sigma_{M_{nominal}} + \sigma_{M_{reconstructed}} \right), 
           M_{nominal} + 5 \times \left( \sigma_{M_{nominal}} + \sigma_{M_{reconstructed}} \right) \right]
$$



In [27]:
M_B0 = [5.27955, 0.00026]
M_K0 = [0.497611, 0.000013]
M_etap = [0.95778, 0.00006]
M_eta = [0.547862, 0.000017]

In [138]:
#Signal = Signal[abs(Signal['B0_M'] - M_B0[0]) < 5 * Signal['B0_ErrM']]
#Signal = Signal[abs(Signal['B0_K_S0_M'] - M_K0[0]) < 5 * Signal['B0_K_S0_ErrM']]
#Signal = Signal[abs(Signal['B0_etap_M'] - M_etap[0]) < 5 * Signal['B0_etap_ErrM']]
#Signal = Signal[abs(Signal['B0_etap_eta_M'] - M_eta[0]) < 5 * Signal['B0_etap_eta_ErrM']]
Signal

,exp_no,run_no,evt_no,B0_K_S0_M,B0_K_S0_ErrM,B0_K_S0_SigM,B0_M,B0_ErrM,B0_SigM,B0_etap_M,...,B0_K_S0_TruthVtxProd,B0_TruthTagVx,B0_TruthTagVy,B0_TruthTagVz,B0_mcTagPDG,B0_TruthDeltaT,B0_CSMVA,B0_TrCSMVA,nCands,iCand
0,0,0,350005,0.500407,0.004492,0.621686,5.299881,0.010961,1.856670,0.956255,...,"[0.004215669, -0.0012281875, 0.017204314]",2.074438e-04,0.000700,0.012328,-511,0.630846,0.929897,2.585103,1,0
1,0,0,350024,0.496696,0.004302,-0.213392,5.288764,0.034654,0.266445,0.959869,...,"[0.0014324387, -0.0018377373, 0.020976264]",2.209126e-03,0.003190,0.028776,-511,-0.910539,0.900429,2.202004,8,2
2,0,0,350026,0.501817,0.004086,1.028760,5.303726,0.024997,0.967938,0.962386,...,"[0.0101938, -0.0026822484, 0.03456243]",1.841891e-04,0.000577,0.006411,511,3.412339,0.955028,3.055712,1,0
3,0,0,350034,0.513165,0.011597,1.340991,5.300632,0.029411,0.717458,0.956870,...,"[0.0015784894, 0.0007770268, -0.0049018697]",8.082185e-03,-0.010513,0.041454,-511,-5.444315,0.622517,0.500245,3,2
4,0,0,350037,0.498159,0.002639,0.206475,5.290727,0.014904,0.751298,0.957299,...,"[0.00040547454, 9.8050084e-05, -0.023185262]",8.495884e-04,-0.000629,-0.015444,-511,-0.897916,0.994524,5.201922,1,0
5,0,0,350041,0.497600,0.004264,-0.003369,5.249694,0.030067,-0.992315,0.958983,...,"[-0.0002296408, -0.00027673552, -0.016844284]",1.926204e-03,0.000358,-0.014714,-511,-0.282754,0.660566,0.665817,3,0
6,0,0,350054,0.497564,0.003823,-0.013096,5.279331,0.019945,-0.009969,0.961598,...,"[0.0006155496, -0.0014589654, 0.017896209]",1.292627e-04,0.001460,0.018169,-511,-0.022871,0.803016,1.405250,1,0
7,0,0,350058,0.492344,0.003371,-1.563578,5.255838,0.022732,-1.042220,0.953827,...,"[0.0020724153, 0.0034679456, 0.031267982]",2.774674e-04,-0.000061,-0.009751,511,4.748205,0.579265,0.319759,1,0
8,0,0,350061,0.500738,0.005999,0.520837,5.281927,0.019656,0.121929,0.961539,...,"[0.0027232731, 0.0009872846, -0.019485757]",3.830866e-03,-0.006871,0.006714,-511,-3.032064,0.997623,6.039698,2,0
9,0,0,350062,0.494526,0.002019,-1.530039,5.276699,0.019774,-0.143170,0.950948,...,"[0.009370116, -0.0050246306, 0.022717511]",8.211953e-05,0.007549,0.025921,-511,-0.205774,0.976668,3.734324,1,0


##### Vertex

I need to take a closer look at the vertex information to decide how to use it

In [32]:
# Do stuff

#### Filtering the Signal
We filter the Signal, as we are only interested in correctly reconstructed events

In [142]:
Signal = Signal[Signal['B0_isSignal']==1].reset_index(drop=True)
Signal = Signal.reset_index(drop=True)

#### Some problems (?) with Signal events
- After filtering events for *'B0_isSignal'==1* we should only be left with correctly reconstructed B0 mesons. 
- If I am not mistaken 'Signal' events were created requiring that one side decays in our channel and the other side generically. 
- Given the low branching ratio of our decay channel we would expect close to zero events where both B0 are a Signal

In [144]:
Duplicated = Signal.duplicated(subset='evt_no', keep=False)
Signal[Duplicated]

,exp_no,run_no,evt_no,B0_K_S0_M,B0_K_S0_ErrM,B0_K_S0_SigM,B0_M,B0_ErrM,B0_SigM,B0_etap_M,...,B0_K_S0_TruthVtxProd,B0_TruthTagVx,B0_TruthTagVy,B0_TruthTagVz,B0_mcTagPDG,B0_TruthDeltaT,B0_CSMVA,B0_TrCSMVA,nCands,iCand
424,0,0,353187,0.494744,0.009890,-0.290146,5.260913,0.018450,-1.009037,0.959055,...,"[0.0021385767, -0.0013396078, 0.035637666]",-7.469372e-04,0.002435,0.041356,-511,-0.607045,0.096987,-2.231159,4,1
425,0,0,353187,0.494744,0.009890,-0.290146,5.258888,0.019766,-1.044282,0.975954,...,"[0.0021385767, -0.0013396078, 0.035637666]",-7.469372e-04,0.002435,0.041356,-511,-0.607045,0.060995,-2.734021,4,3
456,0,0,353453,0.489981,0.017502,-0.436136,5.240426,0.016531,-2.365429,0.956344,...,"[0.002550842, -0.002053002, 0.051685933]",3.196333e-05,0.000864,0.046106,-511,0.685683,0.949861,2.941510,2,0
457,0,0,353453,0.500238,0.003900,0.672861,5.277877,0.015905,-0.103909,0.956344,...,"[0.002550842, -0.002053002, 0.051685933]",3.196333e-05,0.000864,0.046106,-511,0.685683,0.947152,2.886040,2,1
527,0,0,353931,0.491868,0.006125,-0.938067,5.218185,0.038410,-1.597084,0.947087,...,"[0.0007882535, -0.00074066897, 0.0054820804]",4.135851e-04,0.000510,0.004914,511,0.071892,0.401995,-0.397161,2,0
528,0,0,353931,0.498055,0.005247,0.083953,5.233146,0.021688,-2.138672,0.947087,...,"[0.0007882535, -0.00074066897, 0.0054820804]",4.135851e-04,0.000510,0.004914,511,0.071892,0.663563,0.679213,2,1
539,0,0,354001,0.490932,0.007685,-0.869518,5.188080,0.013865,-6.595936,0.969519,...,"[0.00012383837, -0.0003888245, 0.009973581]",1.059508e-02,0.000735,0.023538,-511,-1.743024,0.781300,1.273261,2,0
540,0,0,354001,0.498109,0.020667,0.023935,5.193855,0.009767,-8.771778,0.969519,...,"[0.00012383837, -0.0003888245, 0.009973581]",1.059508e-02,0.000735,0.023538,-511,-1.743024,0.825000,1.550594,2,1
668,0,0,354938,0.498071,0.005135,0.089006,5.308878,0.025536,1.149296,0.960094,...,"[0.00065474084, -0.0002614839, -0.001608493]",6.169805e-04,0.000008,-0.003015,-511,0.162345,0.855923,1.781832,6,2
669,0,0,354938,0.498071,0.005135,0.089006,5.304738,0.025439,0.990937,0.956699,...,"[0.00065474084, -0.0002614839, -0.001608493]",6.169805e-04,0.000008,-0.003015,-511,0.162345,0.870768,1.907762,6,5


In [139]:
print(Signal['evt_no'].nunique()) 


19825


In [140]:
Signal

,exp_no,run_no,evt_no,B0_K_S0_M,B0_K_S0_ErrM,B0_K_S0_SigM,B0_M,B0_ErrM,B0_SigM,B0_etap_M,...,B0_K_S0_TruthVtxProd,B0_TruthTagVx,B0_TruthTagVy,B0_TruthTagVz,B0_mcTagPDG,B0_TruthDeltaT,B0_CSMVA,B0_TrCSMVA,nCands,iCand
0,0,0,350005,0.500407,0.004492,0.621686,5.299881,0.010961,1.856670,0.956255,...,"[0.004215669, -0.0012281875, 0.017204314]",2.074438e-04,0.000700,0.012328,-511,0.630846,0.929897,2.585103,1,0
1,0,0,350024,0.496696,0.004302,-0.213392,5.288764,0.034654,0.266445,0.959869,...,"[0.0014324387, -0.0018377373, 0.020976264]",2.209126e-03,0.003190,0.028776,-511,-0.910539,0.900429,2.202004,8,2
2,0,0,350026,0.501817,0.004086,1.028760,5.303726,0.024997,0.967938,0.962386,...,"[0.0101938, -0.0026822484, 0.03456243]",1.841891e-04,0.000577,0.006411,511,3.412339,0.955028,3.055712,1,0
3,0,0,350034,0.513165,0.011597,1.340991,5.300632,0.029411,0.717458,0.956870,...,"[0.0015784894, 0.0007770268, -0.0049018697]",8.082185e-03,-0.010513,0.041454,-511,-5.444315,0.622517,0.500245,3,2
4,0,0,350037,0.498159,0.002639,0.206475,5.290727,0.014904,0.751298,0.957299,...,"[0.00040547454, 9.8050084e-05, -0.023185262]",8.495884e-04,-0.000629,-0.015444,-511,-0.897916,0.994524,5.201922,1,0
5,0,0,350041,0.497600,0.004264,-0.003369,5.249694,0.030067,-0.992315,0.958983,...,"[-0.0002296408, -0.00027673552, -0.016844284]",1.926204e-03,0.000358,-0.014714,-511,-0.282754,0.660566,0.665817,3,0
6,0,0,350054,0.497564,0.003823,-0.013096,5.279331,0.019945,-0.009969,0.961598,...,"[0.0006155496, -0.0014589654, 0.017896209]",1.292627e-04,0.001460,0.018169,-511,-0.022871,0.803016,1.405250,1,0
7,0,0,350058,0.492344,0.003371,-1.563578,5.255838,0.022732,-1.042220,0.953827,...,"[0.0020724153, 0.0034679456, 0.031267982]",2.774674e-04,-0.000061,-0.009751,511,4.748205,0.579265,0.319759,1,0
8,0,0,350061,0.500738,0.005999,0.520837,5.281927,0.019656,0.121929,0.961539,...,"[0.0027232731, 0.0009872846, -0.019485757]",3.830866e-03,-0.006871,0.006714,-511,-3.032064,0.997623,6.039698,2,0
9,0,0,350062,0.494526,0.002019,-1.530039,5.276699,0.019774,-0.143170,0.950948,...,"[0.009370116, -0.0050246306, 0.022717511]",8.211953e-05,0.007549,0.025921,-511,-0.205774,0.976668,3.734324,1,0


In [129]:
FindStuff = Signal[Signal.duplicated(['evt_no'])]
FindStuff.sort_values(by=['evt_no'])
FindStuff = FindStuff.reset_index(drop=True)
FindStuff

,exp_no,run_no,evt_no,B0_K_S0_M,B0_K_S0_ErrM,B0_K_S0_SigM,B0_M,B0_ErrM,B0_SigM,B0_etap_M,...,B0_K_S0_TruthVtxProd,B0_TruthTagVx,B0_TruthTagVy,B0_TruthTagVz,B0_mcTagPDG,B0_TruthDeltaT,B0_CSMVA,B0_TrCSMVA,nCands,iCand
0,0,0,353187,0.494744,0.009890,-0.290146,5.258888,0.019766,-1.044282,0.975954,...,"[0.0021385767, -0.0013396078, 0.035637666]",-7.469372e-04,0.002435,0.041356,-511,-0.607045,0.060995,-2.734021,4,3
1,0,0,353453,0.500238,0.003900,0.672861,5.277877,0.015905,-0.103909,0.956344,...,"[0.002550842, -0.002053002, 0.051685933]",3.196333e-05,0.000864,0.046106,-511,0.685683,0.947152,2.886040,2,1
2,0,0,354938,0.498071,0.005135,0.089006,5.304738,0.025439,0.990937,0.956699,...,"[0.00065474084, -0.0002614839, -0.001608493]",6.169805e-04,0.000008,-0.003015,-511,0.162345,0.870768,1.907762,6,5
3,0,0,103222,0.497900,0.007381,0.038694,5.291634,0.020023,0.604510,0.953623,...,"[0.0009443493, -0.00047677837, 0.017901957]",-7.381595e-04,0.000144,0.012937,511,0.600346,0.920964,2.455521,6,4
4,0,0,145609,0.495192,0.005868,-0.412637,5.314476,0.025054,1.394798,0.969771,...,"[-0.0009255678, 0.00063605356, 0.041221157]",1.532602e-03,-0.000067,0.017488,511,2.686072,0.459556,-0.162131,2,1
5,0,0,146047,0.503822,0.003763,1.649634,5.274024,0.015305,-0.359762,0.961721,...,"[0.006891478, 0.009676082, 0.06422835]",1.688044e-04,-0.000373,0.002851,511,7.175413,0.201820,-1.374957,4,2
6,0,0,146955,0.505422,0.062211,0.125508,5.336798,0.029627,1.932958,0.948594,...,"[0.0011147481, -1.8250672e-05, -0.008618884]",2.883015e-03,0.000152,-0.004504,-511,-0.504115,0.657245,0.651042,2,1
7,0,0,148238,0.489118,0.084134,-0.100977,5.261604,0.029231,-0.613267,0.965882,...,"[0.00089831126, -0.000114978226, -0.0045571467]",-4.037681e-04,0.000475,0.005932,-511,-1.183375,0.710475,0.897691,4,2
8,0,0,148773,0.495679,0.003575,-0.541332,5.248227,0.013357,-2.343539,0.967769,...,"[0.008216782, -0.00045833542, 0.013579728]",1.224247e-03,0.000026,-0.005392,511,2.303999,0.880185,1.994180,2,1
9,0,0,316230,0.505095,0.010534,0.710244,5.321503,0.008797,4.771435,0.952573,...,"[0.0049436344, -0.0035938777, -0.0065326905]",2.107201e-07,0.000512,-0.022505,-511,1.923240,0.720355,0.946221,2,1


In [108]:
Signal

,exp_no,run_no,evt_no,B0_K_S0_M,B0_K_S0_ErrM,B0_K_S0_SigM,B0_M,B0_ErrM,B0_SigM,B0_etap_M,...,B0_K_S0_TruthVtxProd,B0_TruthTagVx,B0_TruthTagVy,B0_TruthTagVz,B0_mcTagPDG,B0_TruthDeltaT,B0_CSMVA,B0_TrCSMVA,nCands,iCand
0,0,0,350005,0.500407,0.004492,0.621686,5.299881,0.010961,1.856670,0.956255,...,"[0.004215669, -0.0012281875, 0.017204314]",2.074438e-04,0.000700,0.012328,-511,0.630846,0.929897,2.585103,1,0
1,0,0,350024,0.496696,0.004302,-0.213392,5.288764,0.034654,0.266445,0.959869,...,"[0.0014324387, -0.0018377373, 0.020976264]",2.209126e-03,0.003190,0.028776,-511,-0.910539,0.900429,2.202004,8,2
2,0,0,350026,0.501817,0.004086,1.028760,5.303726,0.024997,0.967938,0.962386,...,"[0.0101938, -0.0026822484, 0.03456243]",1.841891e-04,0.000577,0.006411,511,3.412339,0.955028,3.055712,1,0
3,0,0,350034,0.513165,0.011597,1.340991,5.300632,0.029411,0.717458,0.956870,...,"[0.0015784894, 0.0007770268, -0.0049018697]",8.082185e-03,-0.010513,0.041454,-511,-5.444315,0.622517,0.500245,3,2
4,0,0,350037,0.498159,0.002639,0.206475,5.290727,0.014904,0.751298,0.957299,...,"[0.00040547454, 9.8050084e-05, -0.023185262]",8.495884e-04,-0.000629,-0.015444,-511,-0.897916,0.994524,5.201922,1,0
5,0,0,350041,0.497600,0.004264,-0.003369,5.249694,0.030067,-0.992315,0.958983,...,"[-0.0002296408, -0.00027673552, -0.016844284]",1.926204e-03,0.000358,-0.014714,-511,-0.282754,0.660566,0.665817,3,0
6,0,0,350054,0.497564,0.003823,-0.013096,5.279331,0.019945,-0.009969,0.961598,...,"[0.0006155496, -0.0014589654, 0.017896209]",1.292627e-04,0.001460,0.018169,-511,-0.022871,0.803016,1.405250,1,0
7,0,0,350058,0.492344,0.003371,-1.563578,5.255838,0.022732,-1.042220,0.953827,...,"[0.0020724153, 0.0034679456, 0.031267982]",2.774674e-04,-0.000061,-0.009751,511,4.748205,0.579265,0.319759,1,0
8,0,0,350061,0.500738,0.005999,0.520837,5.281927,0.019656,0.121929,0.961539,...,"[0.0027232731, 0.0009872846, -0.019485757]",3.830866e-03,-0.006871,0.006714,-511,-3.032064,0.997623,6.039698,2,0
9,0,0,350062,0.494526,0.002019,-1.530039,5.276699,0.019774,-0.143170,0.950948,...,"[0.009370116, -0.0050246306, 0.022717511]",8.211953e-05,0.007549,0.025921,-511,-0.205774,0.976668,3.734324,1,0


In [109]:
print(18806-18701)

105
